
# ⚛️ Quantum Portfolio Optimizer

_Date generated: 2025-09-03_

This notebook demonstrates how to use **quantum-inspired optimization** for portfolio allocation.

**Contents:**
- Generate/load asset return & covariance data
- Classical Markowitz efficient frontier
- QUBO formulation of mean-variance optimization
- Solve using **dimod** simulated annealer (D-Wave compatible)
- Compare vs classical optimizers
- Visualize allocations & efficient frontier


## 0) Parameters

In [ ]:

PATH_RETURNS = "data/returns.csv"
N_ASSETS = 8
N_DAYS = 250

RISK_AVERSION = 0.5  # lambda for risk-return tradeoff

SEED = 42


## 1) Setup

In [ ]:

import os, math, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
pd.options.display.float_format = "{:,.4f}".format

try:
    import dimod
    HAS_DIMOD = True
except ImportError:
    HAS_DIMOD = False


## 2) Load or Simulate Data

In [ ]:

def load_returns(path=PATH_RETURNS, n_assets=N_ASSETS, n_days=N_DAYS, seed=SEED):
    if os.path.exists(path):
        df = pd.read_csv(path, parse_dates=["date"]).set_index("date")
        return df
    rng = np.random.default_rng(seed)
    dates = pd.bdate_range("2023-01-01", periods=n_days)
    mu = rng.normal(0.0005, 0.0002, n_assets)
    cov = np.diag(rng.uniform(0.0001,0.0003,n_assets))
    data = rng.multivariate_normal(mu, cov, size=n_days)
    cols = [f"A{i}" for i in range(n_assets)]
    return pd.DataFrame(data, index=dates, columns=cols)

rets = load_returns()
mu = rets.mean().values
cov = rets.cov().values

rets.head()


## 3) Classical Mean-Variance Frontier

In [ ]:

from scipy.optimize import minimize

def port_stats(w, mu, cov):
    r = np.dot(w, mu)
    v = np.dot(w, np.dot(cov, w))
    return r, v, r/np.sqrt(v) if v>0 else np.nan

def solve_classical(mu, cov, lmbd=RISK_AVERSION):
    n = len(mu)
    cons = ({'type':'eq','fun':lambda w: np.sum(w)-1})
    bounds = [(0,1)]*n
    def obj(w): return -(np.dot(w,mu) - lmbd*np.dot(w,np.dot(cov,w)))
    res = minimize(obj, np.ones(n)/n, bounds=bounds, constraints=cons)
    return res.x if res.success else np.ones(n)/n

w_classical = solve_classical(mu,cov)
port_stats(w_classical, mu, cov)


## 4) QUBO Formulation

In [ ]:

def build_qubo(mu, cov, lmbd=RISK_AVERSION, scale=100):
    n = len(mu)
    Q = {}
    for i in range(n):
        Q[(i,i)] = -mu[i]*scale + lmbd*cov[i,i]*scale
        for j in range(i+1,n):
            Q[(i,j)] = lmbd*cov[i,j]*scale
    return Q

Q = build_qubo(mu,cov)
list(Q.items())[:5]


## 5) Quantum-Inspired Solution (dimod annealer)

In [ ]:

if HAS_DIMOD:
    sampler = dimod.SimulatedAnnealingSampler()
    sampleset = sampler.sample_qubo(Q, num_reads=200)
    best = sampleset.first.sample # type: ignore
    w_quantum = np.array([best[i] for i in range(len(mu))],dtype=float)
    if w_quantum.sum()>0:
        w_quantum = w_quantum/w_quantum.sum()
else:
    w_quantum = np.ones(len(mu))/len(mu)

port_stats(w_quantum, mu, cov)


## 6) Compare Allocations

In [ ]:

df_alloc = pd.DataFrame({
    "Classical": w_classical,
    "Quantum": w_quantum
}, index=[f"A{i}" for i in range(len(mu))])
df_alloc


In [ ]:

df_alloc.plot(kind="bar", figsize=(10,4))
plt.title("Portfolio Allocations: Classical vs Quantum-Inspired")
plt.show()
